In [18]:
import os
from dotenv import load_dotenv, dotenv_values
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool,initialize_agent
from langchain.agents.types import AgentType
load_dotenv()

True

In [19]:
system_message = """
"You are the XYZ bot."
"This is conversation with a human. Answer the questions you get based on the knowledge you have."
"If you don't know the answer, just say that you don't, don't try to make up an answer."
"""

In [21]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # Name of the language model
    temperature=0  # Parameter that controls the randomness of the generated responses
)

In [22]:
embeddings = OpenAIEmbeddings()

c:\Users\victo\Desktop\data\data-science-snippets\env-data\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [86]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("ken127322.pdf")
doc = loader.load()


In [91]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators= "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
)
text_chunks = text_splitter.split_documents(doc)

In [93]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [94]:
retriever = vectorstore.as_retriever()

In [95]:
docs = retriever.get_relevant_documents("who is a president")

In [118]:
docs[0].metadata['source']
docs[0].metadata['page']
docs[0].page_content

'with the principle of service to the people of Kenya, and for their well -\nbeing and benefit.  \nThe National Executive.  \n130. (1) The national executi ve of the Republic comprises the \nPresident, the Deputy President and the rest of the Cabinet.  \n \n(2) The composition of the national executive shall reflect the \nregional and ethnic diversity of the people of Kenya.  \nPART 2—THE PRESIDENT AND DEPUTY PRESIDENT  \nAuthority of the President.  \n131. (1) The President — \n(a) is the Head of State and Government;  \n(b) exercises the executive authority of the Republic, with the \nassistance of the Deputy President and Cabinet Secretaries;  \n(c) is the Commander -in-Chief of the Ke nya Defence Forces;  \n(d) is the chairperson of the National Security Council; and  \n(e) is a symbol of national unity.'

In [97]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key='input', return_messages=True, output_key='output')

In [59]:
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        verbose=True,
        return_source_documents=True
    )

In [99]:
tools = [
        Tool(
            name="doc_search_tool",
            func=qa,
            description=(
               "This tool is used to retrieve information from the knowledge base"
            )
        )
    ]

In [101]:
agent = initialize_agent(
        agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        tools=tools,
        llm=llm,
        memory=memory,
        return_source_documents=True,
        return_intermediate_steps=True,
        agent_kwargs={"system_message": system_message}
        )

In [102]:
query1 = "how to appoint chief justice?"
result1 = agent(query1)

In [103]:
result1['intermediate_steps'][0][1]['source_documents'][0].metadata

IndexError: list index out of range

In [104]:
result1['output']

"The Chief Justice of a country is typically appointed by the head of state or government, following a process outlined in the country's constitution or laws. The specific procedure can vary between different countries."

In [105]:
dict(result1)

{'input': 'how to appoint chief justice?',
 'chat_history': [HumanMessage(content='how to appoint chief justice?'),
  AIMessage(content="The Chief Justice of a country is typically appointed by the head of state or government, following a process outlined in the country's constitution or laws. The specific procedure can vary between different countries.")],
 'output': "The Chief Justice of a country is typically appointed by the head of state or government, following a process outlined in the country's constitution or laws. The specific procedure can vary between different countries.",
 'intermediate_steps': []}

In [71]:
for x in range(len(result1['intermediate_steps'][0][1]["source_documents"][0].metadata)):
    print(result1["source_documents"][x].metadata)